In [42]:
%pylab inline
import geopandas as gpd
from geopandas.tools import sjoin
import pandas as pd
from shapely.geometry import Point,LineString
from IPython.display import display
from pygeocoder import Geocoder
import numpy as np,pickle as pkl
import googlemaps,csv,math,os,re

Populating the interactive namespace from numpy and matplotlib


### Philly Street map

In [2]:
ph_str_gpd = gpd.read_file('phillydata/Street_Centerline_qgis3.geojson')
ph_str_bfr_gpd = ph_str_gpd[['STCL2_ID','SEG_ID','geometry']].copy()
ph_str_bfr_gpd.geometry = ph_str_bfr_gpd.buffer(0.0001)
ph_str_gpd.head().T

,0,1,2,3,4
CLASS,3,3,3,5,4
FNODE_,2,2,1,6,5
LENGTH,449.863,540.083,446.104,447.261,148.216
LPOLY_,0,0,0,0,0
L_F_ADD,1500,400,1600,1600,350
L_HUNDRED,1500,400,1600,1600,300
L_T_ADD,1598,498,1698,1698,398
MULTI_REP,0,0,0,0,0
NEWSEGDATE,None,None,None,None,None
ONEWAY,FT,TF,FT,TF,TF


### Crime data

In [11]:
crime_gpd = gpd.read_file('phillydata/Crime.geojson')
print(crime_gpd.shape)
crime_gpd.head().T

(2188216, 19)


,0,1,2,3,4
:@computed_region_bbgf_pidf,None,None,None,20,8
:created_at,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23
:id,row-cgp2~ximi-w3kc,row-hrjd_fgm5-iewt,row-ppi4-axzg~7zk6,row-hf25.7fjs_eu3y,row-qfda-hs3n_qwhp
:updated_at,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43
:version,rv-mcz3-b9uh~3mz2,rv-2rv5_a3zn~htkb,rv-7fh9~wy9e.zkeb,rv-fe39_bach.bvvu,rv-x4re.dktk~tjjt
dc_dist,18,14,25,35,09
dc_key,200918067518,200914033994,200925083199,200935061008,200909030511
dispatch_date,2009-10-02,2009-05-10,2009-08-07,2009-07-19,2009-06-25
dispatch_date_time,2009-10-02T14:24:00,2009-05-10T00:55:00,2009-08-07T15:40:00,2009-07-19T01:09:00,2009-06-25T00:14:00
dispatch_time,14:24:00,00:55:00,15:40:00,01:09:00,00:14:00


### Crime data for 2014-2016

In [23]:
temp1 = crime_gpd[crime_gpd['dispatch_date'].str.contains('2014', na=False)]
temp2 = crime_gpd[crime_gpd['dispatch_date'].str.contains('2015', na=False)]
temp3 = crime_gpd[crime_gpd['dispatch_date'].str.contains('2016', na=False)]
temp = pd.concat([temp1,temp2,temp3])
crime_gpd = temp[~temp['point_x'].isnull()]
crime_gpd.reset_index(drop=True,inplace=True)
crime_gpd.to_pickle('phillydata/Crime.pkl')
crime_gpd.head().T


,0,1,2,3,4
:@computed_region_bbgf_pidf,1,3,2,2,2
:created_at,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23,2016-04-21T18:56:23
:id,row-dpfn-k736~bpy3,row-zxgm.7rtj_ahft,row-fpp8-bhtp-btm6,row-kgu8~cahg.jv6z,row-fmiw.q9hq.3j4d
:updated_at,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43,2016-04-21T20:16:43
:version,rv-dzah~xuaw~s3qx,rv-6gci.n5si~2r66,rv-8mr4-b9bu~neqi,rv-gsxm.pxdv~n72t,rv-d9j5~nise_dazc
dc_dist,01,03,02,02,02
dc_key,201401011453,201403030753,201402020489,201402038356,201402056495
dispatch_date,2014-03-17,2014-06-01,2014-04-09,2014-06-26,2014-09-16
dispatch_date_time,2014-03-17T16:37:00,2014-06-01T15:00:00,2014-04-09T17:40:00,2014-06-26T14:02:00,2014-09-16T18:15:00
dispatch_time,16:37:00,15:00:00,17:40:00,14:02:00,18:15:00


### Spatial join of Crime data

In [4]:
buf = 0.0001
crime_bfr_gpd = crime_gpd[['objectid','text_general_code','dispatch_date','geometry']].copy()
crime_bfr_gpd.geometry = crime_bfr_gpd.buffer(buf)
joined = sjoin(crime_bfr_gpd, ph_str_bfr_gpd, how='left', op='intersects')
joined.dispatch_date = pd.to_datetime(joined.dispatch_date)
gb = joined.reset_index(drop=True).groupby('objectid').count()
print('%d points spatially match no segment' % gb[gb.index_right==0].shape[0])
print('%d points spatially match 1 segment' % gb[gb.index_right==1].shape[0])
print('%d points spatially match multiple segments' % gb[gb.index_right>1].shape[0])
gb.columns.values

7 points spatially match no segment
188237 points spatially match 1 segment
329444 points spatially match multiple segments


array(['text_general_code', 'dispatch_date', 'geometry', 'index_right',
       'STCL2_ID', 'SEG_ID'], dtype=object)

In [24]:
joined['mmyy'] = joined['dispatch_date'].apply(lambda x: x.strftime('%Y-%m'))     
joined.head().T

,0,0,0,1,1
objectid,1850353,1850353,1850353,1852448,1852448
text_general_code,Thefts,Thefts,Thefts,Thefts,Thefts
dispatch_date,2014-03-17 00:00:00,2014-03-17 00:00:00,2014-03-17 00:00:00,2014-06-01 00:00:00,2014-06-01 00:00:00
geometry,"POLYGON ((-75.17791799999999 39.920288, -75.17...","POLYGON ((-75.17791799999999 39.920288, -75.17...","POLYGON ((-75.17791799999999 39.920288, -75.17...","POLYGON ((-75.15180599999999 39.929027, -75.15...","POLYGON ((-75.15180599999999 39.929027, -75.15..."
index_right,4762,4763,4765,12996,14016
STCL2_ID,192,163,193,5912,5905
SEG_ID,220192,220163,220193,341222,341215
month,3,3,3,6,6
year,2014,2014,2014,2014,2014
mmyy,2014-03,2014-03,2014-03,2014-06,2014-06


In [57]:
gb2 = joined.groupby(['SEG_ID','mmyy']).size().reset_index()
gb2.columns = ['STREETSEGID','DATE','COUNT']
crime_stats = pd.Series(gb2.COUNT.values,index=[gb2.STREETSEGID,gb2.DATE]).to_dict()
f = open('tripbytrip_geojsonCyclingPhilly/Crime_Stats.pkl','wb')
pkl.dump(crime_stats,f)
f = open('tripbytrip_geojsonCyclingPhilly/Crime_Stats.pkl','rb')
crime_stats = pkl.load(f)
crime_stats
# gb2.set_index(['STREETSEGID','DATE'],inplace=True)
# uid = gb2.index.unique()
# crime_statistic = pd.DataFrame(index=range(len(uid)),columns=['STREETSEGID','MONTH','YEAR'])

# for idx,ids in enumerate(uid):
#     temp = gb2.loc[ids]
#     crime_statistic.set_value(idx,"STREETSEGID",ids[0])
#     crime_statistic.set_value(idx,"MONTH",ids[1])
#     crime_statistic.set_value(idx,"YEAR",ids[2])
#     for index, row in temp.iterrows():
#         crime_statistic.set_value(idx, row['OFFENSE']+"_COUNT",row['COUNT'])
# crime_statistic.fillna(0,inplace=True)    
# crime_statistic['CRIME_COUNT'] = crime_statistic.iloc[:,2:crime_statistic.columns.size].sum(axis=1)
# temp = ph_str_gpd[['SEG_ID','SHAPE_LEN']]
# crime_statistic = crime_statistic.merge(temp,left_on='STREETSEGID',right_on='SEG_ID',right_index=False)
# crime_statistic.rename(columns={'SHAPE_Length': 'LENGTH'}, inplace=True) 

# temp = list(crime_statistic.columns)
# temp.insert(2,'LENGTH')
# del temp[-1]
# crime_statistic = crime_statistic[temp]

# for column in crime_statistic.columns:
#     if('_COUNT' in column):
#         col_len = column + '/LEN'
#         crime_statistic[col_len] = crime_statistic[column]/crime_statistic.LENGTH
        
# crime_statistic.head().T

{(621127.0, '2016-02'): 1,
 (721103.0, '2015-05'): 1,
 (341734.0, '2015-11'): 3,
 (940129.0, '2015-07'): 2,
 (240488.0, '2014-07'): 1,
 (442212.0, '2014-01'): 1,
 (761497.0, '2014-10'): 1,
 (540886.0, '2016-11'): 2,
 (821613.0, '2014-04'): 1,
 (741390.0, '2015-05'): 4,
 (980513.0, '2014-10'): 4,
 (320171.0, '2016-06'): 2,
 (423060.0, '2016-03'): 5,
 (321312.0, '2015-02'): 1,
 (421760.0, '2015-09'): 2,
 (762867.0, '2014-06'): 1,
 (401333.0, '2015-04'): 2,
 (500322.0, '2014-03'): 2,
 (200135.0, '2016-08'): 2,
 (500372.0, '2014-11'): 2,
 (1060095.0, '2016-10'): 1,
 (521216.0, '2016-04'): 1,
 (441107.0, '2014-08'): 1,
 (440955.0, '2015-05'): 2,
 (721686.0, '2014-10'): 2,
 (200286.0, '2014-03'): 2,
 (980070.0, '2014-11'): 1,
 (541558.0, '2014-07'): 4,
 (760151.0, '2014-02'): 1,
 (761520.0, '2015-04'): 1,
 (541275.0, '2014-03'): 2,
 (220645.0, '2015-11'): 2,
 (240560.0, '2014-07'): 5,
 (440463.0, '2014-06'): 3,
 (441168.0, '2016-03'): 1,
 (220610.0, '2015-09'): 1,
 (440129.0, '2016-04'): 1,


In [45]:
temp = ph_str_gpd[['SEG_ID','SHAPE_LEN']]
temp3 = crime_statistic.merge(temp,left_on='STREETSEGID',right_on='SEG_ID',right_index=False)
temp3.rename(columns={'SHAPE_LEN': 'LENGTH'}, inplace=True) 

temp = list(temp3.columns)
print(temp)
temp.insert(2,'LENGTH')
del temp[-1]
temp3 = temp3[temp]
temp3.head().T
for column in temp3.columns:
    if('_COUNT' in column):
        col_len = column + '/LEN'
        temp3[col_len] = temp3[column]/temp3.LENGTH
        
temp3.drop('SEG_ID', axis=1, inplace=True)
crime_statistic = temp3


['STREETSEGID', 'MONTH', 'DRIVING UNDER THE INFLUENCE_COUNT', 'Theft from Vehicle_COUNT', 'Vandalism/Criminal Mischief_COUNT', 'Aggravated Assault Firearm_COUNT', 'Arson_COUNT', 'Recovered Stolen Motor Vehicle_COUNT', 'Aggravated Assault No Firearm_COUNT', 'All Other Offenses_COUNT', 'Thefts_COUNT', 'Fraud_COUNT', 'Other Assaults_COUNT', 'Burglary Residential_COUNT', 'Homicide - Criminal_COUNT', 'Motor Vehicle Theft_COUNT', 'Rape_COUNT', 'Robbery No Firearm_COUNT', 'Weapon Violations_COUNT', 'Other Sex Offenses (Not Commercialized)_COUNT', 'Robbery Firearm_COUNT', 'Embezzlement_COUNT', 'Burglary Non-Residential_COUNT', 'Narcotic / Drug Law Violations_COUNT', 'Disorderly Conduct_COUNT', 'Prostitution and Commercialized Vice_COUNT', 'Receiving Stolen Property_COUNT', 'Gambling Violations_COUNT', 'Liquor Law Violations_COUNT', 'Forgery and Counterfeiting_COUNT', 'Offenses Against Family and Children_COUNT', 'Vagrancy/Loitering_COUNT', 'Public Drunkenness_COUNT', 'Homicide - Gross Negligen

In [46]:
crime_statistic.to_csv('phillydata/Crime_Details.csv',index=False)

In [4]:
# park_gpd = gpd.read_file('phillydata/Parking_Violations_2016.csv')
# print(crime_gpd.shape)
# crime_gpd.head().T

data = pd.DataFrame.from_csv('phillydata/Parking_Violations_2014.csv',index_col=False)
temp = data[~data['Coordinates'].isnull()]
temp3 = list(map(lambda x: [float(i) for i in x.strip()[1:-1].split(",")], temp['Coordinates']))
points = [Point(sub[1],sub[0]) for sub in temp3]
park_gpd = gpd.GeoDataFrame(temp,geometry=points)
park_gpd.head().T

,2,5,6,7,8
Unnamed: 0,3,6,7,8,9
Issue Date and Time,02/21/2015 12:40:00 PM,03/10/2015 12:15:00 PM,03/11/2015 12:50:00 PM,03/18/2015 02:00:00 PM,03/22/2015 06:40:00 PM
State,PA,PA,PA,PA,AL
Plate ID,1554828,1532880,1610648,1405050,1088
Division,0,0,0,0,0
Location,4907 N 5TH,400 E CAMBRI,2500 S 63RD,1041 WOLF ST,5300 OAKLAND
Location Standardized,"4907 N 5TH ST, 19120","400 E CAMBRIA ST, 19134","2500 S 63RD ST, 19142","1041 WOLF ST, 19148","5300 OAKLAND ST, 19124"
Coordinates,"(40.0265298416,-75.1326747799)","(39.9931206598,-75.1237010395)","(39.9237189747,-75.227342061)","(39.9211816852,-75.1647196738)","(40.0269171663,-75.0807203956)"
Violation Description,DOUBLE PARKED,FIRE HYDRANT,DOUBLE PARKED,PARKING PROHIBITED,DOUBLE PARKED
Fine,$51.00,$76.00,$51.00,$31.00,$51.00


In [ ]:
buf = 0.0001
park_gpd.crs = ph_str_gpd.crs
park_bfr_gpd = park_gpd[['Unnamed: 0','Violation Description','Issue Date and Time','geometry']].copy()
park_bfr_gpd.geometry = park_bfr_gpd.buffer(buf)
joined1 = sjoin(park_bfr_gpd, ph_str_bfr_gpd, how='left', op='intersects')
print(joined1.columns.values)
joined1.head().T

# joined.dispatch_date = pd.to_datetime(joined.dispatch_date).dt.month
# gb = joined.reset_index().groupby('objectid').count()
# print('%d points spatially match no segment' % gb[gb.index_right==0].shape[0])
# print('%d points spatially match 1 segment' % gb[gb.index_right==1].shape[0])
# print('%d points spatially match multiple segments' % gb[gb.index_right>1].shape[0])
# gb.columns.values

Exception ignored in: <bound method PreparedGeometry.__del__ of <shapely.prepared.PreparedGeometry object at 0x13a7e1f98>>
Traceback (most recent call last):
  File "//anaconda/lib/python3.5/site-packages/shapely/prepared.py", line 32, in __del__
    lgeos.GEOSPreparedGeom_destroy(self.__geom__)
KeyboardInterrupt: 
Exception ignored in: <bound method PreparedGeometry.__del__ of <shapely.prepared.PreparedGeometry object at 0x13acbb048>>
Traceback (most recent call last):
  File "//anaconda/lib/python3.5/site-packages/shapely/prepared.py", line 32, in __del__
    lgeos.GEOSPreparedGeom_destroy(self.__geom__)
KeyboardInterrupt: 


In [82]:
joined.head().T

,1,2,3,5,6
Unnamed: 0,2,3,4,6,7
Violation Description,DOUBLE PARKED,BUS ONLY ZONE,BUS ONLY ZONE,PARKING PROHIBITED,PARKING PROHIBITED
Issue Date and Time,07/01/2014 12:36:00 PM,07/21/2014 04:37:00 AM,03/31/2014 06:05:00 PM,04/05/2014 03:30:00 PM,04/05/2014 03:30:00 PM
geometry,"POLYGON ((-75.123023815 39.9911330252, -75.123...","POLYGON ((-75.15283649049999 40.0110985867, -7...","POLYGON ((-75.0830514148 40.0173483738, -75.08...","POLYGON ((-75.1704781688 39.9542887577, -75.17...","POLYGON ((-75.1704781688 39.9542887577, -75.17..."
index_right,15531,25299,8580,500,500
STCL2_ID,919,5933,12576,85863,85863
SEG_ID,540919,620395,660337,421373,421373


In [81]:
final = pd.concat([joined,joined1])
final['Issue Date and Time'] = pd.to_datetime(final['Issue Date and Time']).dt.month
final.head().T

,1,2,3,5,6
Unnamed: 0,2,3,4,6,7
Violation Description,DOUBLE PARKED,BUS ONLY ZONE,BUS ONLY ZONE,PARKING PROHIBITED,PARKING PROHIBITED
Issue Date and Time,7,7,3,4,4
geometry,"POLYGON ((-75.123023815 39.9911330252, -75.123...","POLYGON ((-75.15283649049999 40.0110985867, -7...","POLYGON ((-75.0830514148 40.0173483738, -75.08...","POLYGON ((-75.1704781688 39.9542887577, -75.17...","POLYGON ((-75.1704781688 39.9542887577, -75.17..."
index_right,15531,25299,8580,500,500
STCL2_ID,919,5933,12576,85863,85863
SEG_ID,540919,620395,660337,421373,421373


In [87]:
# joined = final.copy()
# gb2 = joined.groupby(['SEG_ID','Issue Date and Time','Violation Description']).size().reset_index()
# gb2.columns = ['STREETSEGID','MONTH','TYPE','COUNT']
# gb2.set_index(['STREETSEGID','MONTH'],inplace=True)
# uid = gb2.index.unique()
# park_statistic = pd.DataFrame(index=range(len(uid)),columns=['STREETSEGID','MONTH'])

# for idx,ids in enumerate(uid):
#     temp = gb2.loc[ids]
#     park_statistic.set_value(idx,"STREETSEGID",ids[0])
#     park_statistic.set_value(idx,"MONTH",ids[1])
#     for index, row in temp.iterrows():
#         park_statistic.set_value(idx, row['TYPE']+"_COUNT",row['COUNT'])
# park_statistic.fillna(0,inplace=True)    
# park_statistic['TYPE_COUNT'] = park_statistic.iloc[:,2:park_statistic.columns.size].sum(axis=1)
# temp = ph_str_gpd[['SEG_ID','SHAPE_LEN']]
# park_statistic = park_statistic.merge(temp,left_on='STREETSEGID',right_on='SEG_ID',right_index=False)
park_statistic.rename(columns={'SHAPE_LEN': 'LENGTH'}, inplace=True) 

temp = list(park_statistic.columns)
temp.insert(2,'LENGTH')
del temp[-1]
park_statistic = park_statistic[temp]

for column in park_statistic.columns:
    if('_COUNT' in column):
        col_len = column + '/LEN'
        park_statistic[col_len] = park_statistic[column]/park_statistic.LENGTH
        
park_statistic.head().T

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,0,1,2,3,4
STREETSEGID,100003.000000,100008.000000,100009.000000,100009.000000,100009.000000
MONTH,2.000000,9.000000,3.000000,4.000000,8.000000
LENGTH,935.023163,728.727158,281.855359,281.855359,281.855359
EXPIRED INSPECTION_COUNT,1.000000,0.000000,0.000000,1.000000,1.000000
STOPPING PROHIBITED_COUNT,0.000000,1.000000,0.000000,0.000000,0.000000
IMPROPER ON 2WAY HWY_COUNT,0.000000,0.000000,1.000000,1.000000,0.000000
UNREG/ABANDONED VEH_COUNT,0.000000,0.000000,0.000000,0.000000,0.000000
COMMRCL VEH RES AREA_COUNT,0.000000,0.000000,0.000000,0.000000,0.000000
SUSPENDED LICENSE_COUNT,0.000000,0.000000,0.000000,0.000000,0.000000
EXPIRED TAG_COUNT,0.000000,0.000000,0.000000,0.000000,0.000000


In [89]:
park_statistic.to_csv('phillydata/Parking_Violation_Details.csv',index=False)

In [3]:
str_gpd = gpd.read_file('Street_Segments/Street_Segments.geojson')
str_bfr_gpd = str_gpd[['OBJECTID','STREETSEGID','geometry','SHAPE_Length']].copy()
str_bfr_gpd.geometry = str_bfr_gpd.buffer(0.0001)
str_gpd.head().T

,0,1,2,3,4
BEGINMEASURE,0,0,0,0,0
DIRECTIONALITY,0,1,1,2,2
ENDMEASURE,192.925,131.936,117.887,108.331,119.19
FACILITYID,SEGID-876,SEGID-13161,SEGID-8240,SEGID-10395,SEGID-2216
FROMADDRESSLEFTTHEO,5000,5200,5300,5400,5500
FROMADDRESSRIGHTTHEO,5001,5203,5301,5401,5501
FROMNODEID,18637,20290,19814,19815,20876
OBJECTID,1,2,3,4,5
OBJECTID_1,37,38,39,40,41
QUADRANT,NW,NW,NW,NW,NW


In [28]:
import calendar
month = list(calendar.month_name)
final = []
for i in range(1,13):
    filename = 'dcdata/Parking Violations/Parking_Violations_in_'+month[i]
    park_gpd1 = gpd.read_file(filename+'_2014.geojson')
    park_gpd2 = gpd.read_file(filename+'_2015.geojson')
    temp = park_gpd1.append(park_gpd2,ignore_index=True).copy()
    temp = temp.groupby(['STREETSEGID','MONTH_OF_YEAR','VIOLATION_DESCRIPTION']).size().reset_index()
    temp.columns = ['STREETSEGID','MONTH','TYPE','COUNT']
    temp.set_index(['STREETSEGID','MONTH'],inplace=True)
    uid = temp.index.unique()
    park_statistic = pd.DataFrame(index=range(len(uid)),columns=['STREETSEGID','MONTH'])

    for idx,ids in enumerate(uid):
        temp1 = temp.loc[ids]
        park_statistic.set_value(idx,"STREETSEGID",ids[0])
        park_statistic.set_value(idx,"MONTH",ids[1])
        for index, row in temp1.iterrows():
            park_statistic.set_value(idx, row['TYPE']+"_COUNT",row['COUNT'])
    park_statistic = park_statistic.reset_index(drop=True)
    if(i==1):
        final = park_statistic.copy()
    else:
        final = final.append(park_statistic,ignore_index=True)
    final = final.reset_index(drop=True)
    print(month[i])
    final.fillna(0,inplace=True)
final.head().T

January
February
March
April
May
June
July
August
September
October
November
December


,0,1,2,3,4
ABANDON VEH PRIV/PUBLIC PROP SOLID WASTE/RAT_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 1_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 2_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 3_COUNT,0,0,0,0,0
CMV- LOADING ZONE W/O ANNUAL OR DAY PASS_COUNT,0,0,0,0,0
CMV- NON LOAD ZONE METER W/O ANNUAL OR DAY PASS_COUNT,0,0,0,0,0
COMM VEH MORE THAN 22 FEET IN FRONT OF RESIDENCE_COUNT,0,0,0,0,0
COMMERCIAL MOTOR VEHICLE - NOT OUT OF SERVICE_COUNT,0,0,0,0,0
COMMERCIAL MOTOR VEHICLE - OUT OF SERVICE_COUNT,0,0,0,0,0
"COMMERCIAL PERMIT PARKING DAY PASS, DISPLAY EXPIRE_COUNT",0,0,0,0,0


In [30]:
park_statistic = final.copy()

In [56]:
# park_statistic['TYPE_COUNT'] = park_statistic.iloc[:,2:park_statistic.columns.size].sum(axis=1)
# temp = str_gpd[['STREETSEGID','SHAPE_Length']]
# park_statistic = park_statistic.merge(temp,left_on='STREETSEGID',right_on='STREETSEGID',right_index=False)
# park_statistic.rename(columns={'SHAPE_Length': 'LENGTH'}, inplace=True) 
# park_statistic.head().T

import re
temp = list(park_statistic.columns.values)
sub = '_COUNT'
temp2 = [s for s in filter (lambda x: sub in x, temp)]
temp = ['STREETSEGID','MONTH','LENGTH']+temp2
park_statistic = park_statistic[temp]

for column in park_statistic.columns:
    if('_COUNT' in column):
        col_len = column + '/LEN'
        park_statistic[col_len] = park_statistic[column]/park_statistic.LENGTH
        
park_statistic.head().T

,0,1,2,3,4
STREETSEGID,1,1,1,1,3
MONTH,1,5,8,9,1
LENGTH,180.261,180.261,180.261,180.261,109.595
ABANDON VEH PRIV/PUBLIC PROP SOLID WASTE/RAT_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 1_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 2_COUNT,0,0,0,0,0
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 3_COUNT,0,0,0,0,0
CMV- LOADING ZONE W/O ANNUAL OR DAY PASS_COUNT,0,0,0,0,0
CMV- NON LOAD ZONE METER W/O ANNUAL OR DAY PASS_COUNT,0,0,0,0,0
COMM VEH MORE THAN 22 FEET IN FRONT OF RESIDENCE_COUNT,0,0,0,0,0


In [57]:
park_statistic.to_csv('dcdata/Parking Violations/Parking_Violations_Details.csv',index=False)

In [3]:
coord = []
for i,line in enumerate(ph_str_gpd.geometry):
    if(type(line).__name__=='MultiLineString'):
        print(i)
        for l in line:
            coord.append(list(l.coords))
    else:
        coord.append(list(line.coords))
coord = [item for l in coord for item in l]
len(coord)

119154

In [70]:
new = pd.read_csv('phillydata/Philly_Elevation_SegId.csv')
new.columns

,0,1,2,3,4
Longitude,-75.163713,-75.165297,-75.163713,-75.163540,-75.163405
Latitude,39.959817,39.960014,39.959817,39.960627,39.961281
Elevation,12.810000,11.680000,12.810000,15.260000,15.080000
SEG_ID,420708.000000,420708.000000,422065.000000,422065.000000,422065.000000


In [4]:
new = pd.DataFrame(coord)
new.columns = ['Longitude','Latitude']
new.head().T

,0,1,2,3,4
Longitude,-75.163713,-75.165297,-75.163713,-75.163540,-75.163405
Latitude,39.959817,39.960014,39.959817,39.960627,39.961281


In [7]:
import requests
import json
from timeit import Timer
import multiprocessing as mproc

if __name__=='__main__':
    error = []
    def apply_fn(row):
        try:
            print(row.name)
            string = "http://nationalmap.gov/epqs/pqs.php?x="+str(row['Longitude'])+"&y="+str(row['Latitude'])+"&units=Meters&output=json"
            resp = requests.get(string)
            temp = resp.json()
#             row.set_value(row.name,'Elevation',temp['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])
            return temp['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']
        except ValueError:
            error.append(row.name)
    def call_apply_fn(df):
        return df.apply(apply_fn,axis=1)

    pool = mproc.Pool(mproc.cpu_count()) 
    results = pool.map(call_apply_fn, np.array_split(new.ix[0:100],mproc.cpu_count()))
    temp = pd.concat(results)

0
51
76
26
52
77
27
1
78
2
53
28
29
54
79
3
55
4
30
80
81
5
31
56
6
82
32
57
83
7
58
33
84
34
8
59
85
35
9
60
86
61
10
36
87
62
11
37
88
63
38
12
89
64
13
39
90
65
14
40
91
66
41
15
92
67
42
16
93
68
43
17
94
44
69
18
95
45
70
19
96
46
71
20
97
47
21
72
98
48
73
22
99
49
74
23
100
50
75
24
25


In [43]:
final = temp.copy()

temp2 = pd.concat([final[0],final[1],final[2],final[3],final[4]])

coord = [len(list(line.coords)) for line in ph_str_gpd.geometry]
print(len(coord))
segid = ph_str_gpd.SEG_ID
temp3 = []
for idx,seg in enumerate(segid):
    temp3.append([seg]*coord[idx])
temp3 = [item for l in temp3 for item in l]

temp2.loc[:,'STREETSEGID'] = pd.Series(temp3,index=temp2.index)

temp2.to_csv('phillydata/Philly_Elevation_SegId.csv',index=False)

In [50]:
temp = pd.read_csv('phillydata/Parking_Violations.csv')
temp.head().T

,0,1,2,3,4
Issue Date and Time,01/01/2012 12:00:00 AM,01/01/2012 12:00:00 AM,01/01/2012 12:00:00 AM,01/01/2012 12:01:00 AM,01/01/2012 12:07:00 AM
State,PA,PA,PA,TX,PA
Plate ID,1048932,679582,1082165,1720567,1172258
Division,0,9,5,2,63
Location,1000 VINE,400 S 25TH,7016 HENRY AVE,300 DEVEREAU,450 N 6TH
Location Standardized,"1000 VINE ST, 19107","400 S 25TH ST, 19146","7016 HENRY AVE, 19128",NaN,"450 N 6TH ST, 19123"
Coordinates,"(39.9568850255,-75.1563742867)","(39.9473721746,-75.1814961216)","(40.0483515634,-75.2185249487)",NaN,"(39.9598882869,-75.1484695504)"
Violation Description,PARKING PROHBITED CC,SCHOOL ZONE,SIDEWALK,BLOCKING DRIVEWAY,PARKING PROHBITED CC
Fine,$51.00,$36.00,$51.00,$51.00,$51.00
Issuing Agency,POLICE,POLICE,POLICE,POLICE,POLICE


In [56]:
max(temp[temp.columns.values[0]])

'12/31/2016 12:59:00 PM'